[Github link](https://github.com/kriskruse/02467-CSS---Assignment1)

All members of the group contributed equally on the completion of the assignment.

# Part 1: Web-scraping to gather data

1. Inspect the HTML of the pages above and use web-scraping to get the set of participants in 2019. Share your code and add comments to guide us through it.

2. How many unique researchers you got in 2019?

3. Explain one or two decisions you took during the web-scraping exercise, for 2019 or any other year. Why did you take this choice? How might your decision impact the final number of authors?

In [3]:
import requests
from bs4 import BeautifulSoup
from bs4.element import NavigableString
import re

# 2019
speakers_2019 = set()
poster_pattern = re.compile(r"(?<=<li>)(.*?)(?=<span>|<br\/>|<\/li>)") # Regex to find properly segment li's
strong = re.compile(r"(<strong>|</strong>)")

poster_session = requests.get("https://2019.ic2s2.org/posters/").content
poster = BeautifulSoup(poster_session)

uls = poster.find_all("ul")[6:8] # The 7th and 8th uls are the elements containing the list of names and presentations

for ul in uls:
    for li in ul:
        if type(li) == NavigableString:
            continue
        match = re.findall(poster_pattern, str(li))
        if match: # if there was a match proceed
            if re.findall(strong, str(match)): # Filter away poorly written list elements (looking at you title in its own li)
                continue
            speakers = match[0].split(", ") # split names at ", "
            split = speakers[-1].split(" and ") # split the last potential pair of names at " and ", if there isn't a pair nothing happens
            if len(split) == 2: # if there was a pair of names it will be split
                speakers = speakers[:-1] + split
            for speaker in speakers:
                speakers_2019.add(speaker) # finally add the names from the li to the set of speakers

Both the days of the poster session are unordered list elements (ul), for which each bullet point is a list item (li). Bullet points with names followed by the title of presentation starts with a \<li> tag and ends at the beginning of a \<span> tag.

There are some inconsistencies which are caught with the alternative endings of the \<li> tags such as the names of the speakers being only within the \<li> tag. Lastly the \<li> tags containing \<strong> implies that it is NOT a set of speaker names but rather a title that has been matched, which is filtered out.

In [4]:
# 2020 speakers
link = "https://docs.google.com/spreadsheets/u/0/d/e/2PACX-1vTX9_1Xftn7D-nSI8X9b7tafr_Z0kAbphKdfZ8qUSU9p-syXNsGPdhHl5ZyTnKKL-T6dCEJqtsrn3wy/pubhtml/sheet?headers=false&gid=181378784"
text = requests.get(link).content
schedule = BeautifulSoup(text)
rows = schedule.find_all("tr")[2:]

speakers_2020 = set()
for row in rows:
    names = row.find_all("td")[2].get_text()
    names = names.split(",")
    for name in names:
        if name: # ensure non-empty string
            speakers_2020.add(name.strip())


In [8]:
# 2021 speakers
speakers_2021 = set()
html_2021 = requests.get("https://easychair.org/smart-program/IC2S2-2021/talk_author_index.html").content
soup2021 = BeautifulSoup(html_2021)
rows = soup2021.find_all("tr")[2:]

for row in rows:
    name = row.find_all("td")[0].get_text()
    if len(name) != 1:
        last_name, first_name = name.split(", ")
        speakers_2021.add(f"{first_name} {last_name}")

speakers = speakers_2019.union(speakers_2020.union(speakers_2021))

# Part 2: Getting data from the Semantic Scholar API

In [ ]:
import time
import tqdm
speakers = list(speakers)
ids = set()

for name in tqdm.tqdm(speakers):
    params = {"query": name, "fields": "papers.authors", "limit": 1}
    response = requests.get("https://api.semanticscholar.org/graph/v1/author/search", params=params).json()
    for author in response.get("data", []):
        if author:
            ids.add(author["authorId"])
    
    
    


 21%|████████████████▏                                                              | 386/1878 [07:21<57:49,  2.33s/it]